In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Statevector

# 1. Deutsch-Jotzsa algoritem

Motivacije je implementirati standardni Deutsch-Jotzsa algoritem, ki nam predstavi idejo zakaj lahko 
kvantni računalniki "premagajo" klasične za določene probleme. Algoritem je bil uveden v Ref [1].

# 1.1 Deutschov problem

Imamo neznano funkcijo $f$, ki vzame kot input en bit $x$=0 ali 1 in vrne ali 0 ali 1.
Funkcija ima lastnost, da je ali konstantna $f(0)=f(1)$ ali uravnotežena $f(0)\neq f(1)$. 
Primer uravnotežene funkcije bi bila
$$f(0)=1 \qquad f(1)=0$$
in primer konstantne
$$f(0)=0 \qquad f(1)=0.$$

Naloga je ugotoviti, če je funkcija uravnotežena ali konstanta s čim manj izvrednotenj same funkcije.
Če bi hoteli ta problem rešiti klasično, bi morali v najslabšem primeru izvrednotiti funkcijo vsaj $2^{n-1}+1$-krat. 

# 1.2 Kvantna rešitev

Kvantni računalnik lahko ta problem reši bolje in preveri več vhodnih vrednosti naenkrat. Funkcijo $f$ moramo
izvrednotimo kot orakelj $U_f$, ki naredi unitarno transformacijo  $|x>|y> \rightarrow |x>|f(x)+y>$

# Različni oraklji

Konstantni orakelj ima dve možnosti:

a) če je f(x)=0, potem ne naredimo nič [opcija=1]

b) če je f(x)=1, potem apliciramo X vrata na kubit v registru 1 [opcija=2]

c) če je f(x)=balance, CNOT vrata [opcija=3]

d) če je f(x)=1, potem zaporedje X, CNOT in X vrata  na 1.  registru [opcija=4]

In [ ]:
# Pripravi vezje za f(x)=1:  2 kubita in 1 bit
qc = QuantumCircuit(2,1)

opcija=2
#Nastavi stanje |01>
qc.x(1)


qc.h(0)
qc.h(1)

if opcija==0:  # Konstantni orakelj f(x)=0
    print(0)
elif opcija==1:  # Konstantni orakelj f(x)=1
    qc.x(1)
elif opcija==2:  # Uravnovešen orakelj f(x)
    qc.cx(0,1)
elif opcija==3:  # Uravnovešen orakelj f(x)
    qc.x(0)
    qc.cx(0,1)
    qc.x(0)


qc.h(0)

qc.measure(0, 0)

qc.draw("mpl")

In [ ]:
from qiskit import transpile 
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

backend = AerSimulator()

# Prvo prevedemo celotno vezje v low-level instrukcije za simulator
qc_compiled = transpile(qc, backend)

# Izvrednotimo vezje in naredimo 2048 ponovitev
job_sim = backend.run(qc_compiled, shots=2048)

# Grab the results from the job.
result_sim = job_sim.result()



counts = result_sim.get_counts(qc_compiled)
print(counts)

plot_histogram(counts)

# 6. Literatura
David Deutsch and Richard Jozsa (1992). "Rapid solutions of problems by quantum computation". Proceedings of the Royal Society of London A.439: 553-558. doi:10.1098/rspa.1992.0167.